In [ ]:
from openai import OpenAI

client = OpenAI(
    # This is the default and can be omitted
    api_key="dummy-key",
    base_url="http://localhost:9000/v2/models/chatgpt/infer"
)

completion = client.chat.completions.create(
    model="gpt-4.1",
    messages=[
      {
        "role": "user",
        "content": "You are a helpful assistant."
      },
      {
      	"role": "assistant",
      	"content": "Hello! How can I help you?"
      },
      {
      	"role": "user",
      	"content": "What is the capital of Romania?"
      }
    ],
)

KeyboardInterrupt: 

In [ ]:
completion.choices[0].message.content

'The capital of Romania is Bucharest.'

In [2]:
!pip install --upgrade openai

  Using cached openai-1.95.1-py3-none-any.whl.metadata (29 kB)
Using cached openai-1.95.1-py3-none-any.whl (755 kB)
